In [42]:
import cv2

import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision

from PIL import Image

import time

In [43]:
capturing = True
FRAME_RATE = 10
prev = 0

In [68]:
class resNet50_custom(torch.nn.Module):
    def __init__(self):
        super(resNet50_custom, self).__init__()
        self.res50 = torchvision.models.resnet50(pretrained = False, progress = True)
        self.res50.fc = torch.nn.Linear(2048, 1)
    def forward(self, input):
        output = self.res50(input)
        return torch.sigmoid(output)

In [69]:
FaceDetection = resNet50_custom()

FaceDetection.load_state_dict(torch.load('weights_resNet50_FaceDetection_VSC.pth'))
FaceDetection.to('cpu')
FaceDetection.eval();


/Users/thierryksstentini/opt/miniforge3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/thierryksstentini/opt/miniforge3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [75]:
def preprocess(img_array):
    preprocessor = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((256,256)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225])
            ])
    
    return preprocessor(img_array)
count = 0
vid = cv2.VideoCapture(0)
while(capturing):
    # Capture the video frame by frame
    time_elapsed = time.time() - prev
    ret, frame_instant = vid.read()
    
    if time_elapsed > 1./FRAME_RATE:
        prev = time.time()
        #img_PIL = Image.fromarray(np.uint8(frame_instant)).convert('RGB')
        count += 1
        frame_preprocess = preprocess(frame_instant)
        FaceDetection.eval()
        with torch.no_grad():
            pred = FaceDetection(frame_preprocess.unsqueeze(0))
        print("Prediction en sortie de sigmoid : ", pred)

        if count > 100:
            break




Prediction en sortie de sigmoid :  tensor([[0.8799]])
Prediction en sortie de sigmoid :  tensor([[0.8713]])
Prediction en sortie de sigmoid :  tensor([[0.8821]])
Prediction en sortie de sigmoid :  tensor([[0.8563]])
Prediction en sortie de sigmoid :  tensor([[0.8974]])
Prediction en sortie de sigmoid :  tensor([[0.8411]])
Prediction en sortie de sigmoid :  tensor([[0.8461]])
Prediction en sortie de sigmoid :  tensor([[0.8640]])
Prediction en sortie de sigmoid :  tensor([[0.5459]])
Prediction en sortie de sigmoid :  tensor([[0.6006]])
Prediction en sortie de sigmoid :  tensor([[0.8828]])
Prediction en sortie de sigmoid :  tensor([[0.8675]])
Prediction en sortie de sigmoid :  tensor([[0.6912]])
Prediction en sortie de sigmoid :  tensor([[0.7630]])
Prediction en sortie de sigmoid :  tensor([[0.8024]])
Prediction en sortie de sigmoid :  tensor([[0.7982]])
Prediction en sortie de sigmoid :  tensor([[0.8695]])
Prediction en sortie de sigmoid :  tensor([[0.8220]])
Prediction en sortie de sigm

In [71]:
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

end = time.time()